In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import  LinearSVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('./train_V2.csv')
data.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [13]:
def prepare(X):
    dct = {}
    for a, b, c in zip(X.matchId, X.damageDealt, range(X.shape[0])):
        if a not in dct:
            dct[a] = []
        dct[a].append((b, c))
    X['dd_place'] = np.zeros(X.shape[0])
    for x in dct.values():
        x.sort()
        for i in range(len(x)):
            X['dd_place'].at[x[i][1]] = i
    X['boosts+heals'] = X['boosts'] + X['heals']
    X['headshot_rate'] = X['headshotKills'] / X['kills']
    X['headshot_rate'] = X['headshot_rate'].fillna(0)
    X['groupId'] = X['groupId'].astype('category')
    X['matchId'] = X['matchId'].astype('category')
    X['groupId_cat'] = X['groupId'].cat.codes
    X['matchId_cat'] = X['matchId'].cat.codes
    X.drop(columns=['groupId', 'matchId', 'Id'], inplace=True)
    X = pd.get_dummies(X, columns=['matchType'])
    X['rankPoints'] = X['rankPoints'].replace(-1, np.nan)
    X['winPoints'] = X['winPoints'].replace(0, np.nan)
    X.fillna(X.mean(), inplace=True)
#     X = StandardScaler().fit_transform(X)
    return X

In [14]:
y_train = data.winPlacePerc
y_train.fillna(y_train.mean())
X_train = prepare(data.copy().drop(axis=1, columns=['winPlacePerc']))
# X_train = X_train.dropna()
# X_train.isna().any()
y_train = y_train.fillna(y_train.mean())
# y_train.isna().any()
# X_train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,dd_place
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.0
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.0
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.0
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.0
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.0


In [6]:
model = RandomForestRegressor(n_jobs=-1)

In [7]:
# -cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error').mean()

0.09649824844452361

In [ ]:
data_test = pd.read_csv('./test_V2.csv')
Id = list(data_test['Id'])
X_test = prepare(data_test.copy())
y_pred = model.fit(X_train, y_train).predict(X_test)
pd.DataFrame({'Id' : Id, 'winPlacePerc' : y_pred}).to_csv('sample_submission.csv', index=False)